<a href="https://www.kaggle.com/code/hikageshinomori/regression-with-abalone-datas?scriptVersionId=174345268" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from datetime import datetime

# Load the data
train_data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

# Label Encoding
label_encoder = LabelEncoder()

# Apply label encoding to all columns
for column in train_data.columns:
    if train_data[column].dtype == 'object':
        train_data[column] = label_encoder.fit_transform(train_data[column])
        test_data[column] = label_encoder.transform(test_data[column])

# Split features and target
X = train_data.drop(['id', 'Rings'], axis=1)
y = train_data['Rings']

# Feature Engineering
X['Volume'] = X['Length'] * X['Diameter'] * X['Height']
test_data['Volume'] = test_data['Length'] * test_data['Diameter'] * test_data['Height']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_data_scaled = scaler.transform(test_data.drop(['id'], axis=1))

# Splitting data for cross-validation manually
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the TensorFlow model
model = Sequential([
    Conv1D(128, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(1, activation='linear')
])

# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Define callbacks
callbacks = [
    EarlyStopping(patience=30),
    ReduceLROnPlateau(factor=0.5, patience=10)
]



2024-04-27 19:25:27.686634: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 19:25:27.686740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 19:25:27.835203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [2]:
# Train the model
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, 
                    validation_data=(X_val.reshape(-1, X_val.shape[1], 1), y_val), 
                    epochs=150, batch_size=128, callbacks=[callbacks, tensorboard_callback], verbose=1)


Epoch 1/150
 26/567 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 101.8557 - mae: 9.5747

I0000 00:00:1714245952.638768      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1714245952.657559      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 79.2200 - mae: 8.2555

W0000 00:00:1714245962.541409      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1714245963.173166      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


567/567 ━━━━━━━━━━━━━━━━━━━━ 24s 21ms/step - loss: 79.1756 - mae: 8.2522 - val_loss: 6.3434 - val_mae: 1.7117 - learning_rate: 5.0000e-04
Epoch 2/150
567/567 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 12.3198 - mae: 2.6306 - val_loss: 4.0285 - val_mae: 1.3567 - learning_rate: 5.0000e-04
Epoch 3/150
567/567 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 8.7078 - mae: 2.1802 - val_loss: 3.9044 - val_mae: 1.3037 - learning_rate: 5.0000e-04
Epoch 4/150
567/567 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 7.1838 - mae: 1.9501 - val_loss: 3.8575 - val_mae: 1.3236 - learning_rate: 5.0000e-04
Epoch 5/150
567/567 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 6.3943 - mae: 1.8335 - val_loss: 3.7919 - val_mae: 1.3068 - learning_rate: 5.0000e-04
Epoch 6/150
567/567 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 5.6763 - mae: 1.7292 - val_loss: 3.7399 - val_mae: 1.2776 - learning_rate: 5.0000e-04
Epoch 7/150
567/567 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 5.2879 - mae: 1.6585 - val_loss: 3.7316 - val_mae: 1.2809 - learn

In [3]:

# Predict with the TensorFlow model
tensorflow_preds = model.predict(test_data_scaled.reshape(-1, test_data_scaled.shape[1], 1)).flatten()

# Calculate RMSE for TensorFlow
tensorflow_rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val.reshape(-1, X_val.shape[1], 1)).flatten()))
print(f'TensorFlow RMSE: {tensorflow_rmse}')

# Create submission file
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'Rings': tensorflow_preds
})

# Save submission file
submission_df.to_csv('submission_tensorflow_cross_val.csv', index=False)

1888/1888 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
567/567 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
TensorFlow RMSE: 1.8861050851380694


Above gave 0.15018 score

In [4]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Conv1D, MaxPooling1D, Flatten, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from datetime import datetime

# Load the data
train_data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

# Label Encoding
label_encoder = LabelEncoder()

# Apply label encoding to all columns
for column in train_data.columns:
    if train_data[column].dtype == 'object':
        train_data[column] = label_encoder.fit_transform(train_data[column])
        test_data[column] = label_encoder.transform(test_data[column])

# Split features and target
X = train_data.drop(['id', 'Rings'], axis=1)
y = train_data['Rings']

# Feature Engineering
X['Volume'] = X['Length'] * X['Diameter'] * X['Height']
test_data['Volume'] = test_data['Length'] * test_data['Diameter'] * test_data['Height']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_data_scaled = scaler.transform(test_data.drop(['id'], axis=1))

# Splitting data for cross-validation manually
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the TensorFlow model with improved architecture
model = Sequential([
    Conv1D(256, kernel_size=5, activation='linear', input_shape=(X_train.shape[1], 1)),
    LeakyReLU(alpha=0.3),
    BatchNormalization(),
    
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    
    Conv1D(128, kernel_size=3, activation='linear'),
    LeakyReLU(alpha=0.3),
    BatchNormalization(),
    
    LSTM(64),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(1, activation='linear')
])

# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Define callbacks
callbacks = [
    EarlyStopping(patience=30),
    ReduceLROnPlateau(factor=0.5, patience=10)
]

# Train the model
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, 
                    validation_data=(X_val.reshape(-1, X_val.shape[1], 1), y_val), 
                    epochs=100, batch_size=128, callbacks=[callbacks, tensorboard_callback], verbose=1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - loss: 80.6657 - mae: 8.2620 - val_loss: 7.1208 - val_mae: 1.6899 - learning_rate: 5.0000e-04
Epoch 2/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 12.8072 - mae: 2.7253 - val_loss: 4.2202 - val_mae: 1.3822 - learning_rate: 5.0000e-04
Epoch 3/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 9.0420 - mae: 2.2275 - val_loss: 3.8927 - val_mae: 1.3324 - learning_rate: 5.0000e-04
Epoch 4/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 7.4614 - mae: 1.9872 - val_loss: 3.8547 - val_mae: 1.3199 - learning_rate: 5.0000e-04
Epoch 5/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 6.3928 - mae: 1.8341 - val_loss: 3.8314 - val_mae: 1.2952 - learning_rate: 5.0000e-04
Epoch 6/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 5.8185 - mae: 1.7401 - val_loss: 3.8376 - val_mae: 1.2936 - learning_rate: 5.0000e-04
Epoch 7/100
567/567 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 5.3801 - mae: 1.6634 - val_loss: 3.8235 - val_m

In [5]:
tensorflow_preds = model.predict(test_data_scaled.reshape(-1, test_data_scaled.shape[1], 1)).flatten()

# Calculate RMSE for TensorFlow
tensorflow_rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val.reshape(-1, X_val.shape[1], 1)).flatten()))
print(f'TensorFlow RMSE: {tensorflow_rmse}')

# Create submission file
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'Rings': tensorflow_preds
})

# Save submission file
submission_df.to_csv('submission_tensorflow_cross_val_improved.csv', index=False)

1888/1888 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
567/567 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
TensorFlow RMSE: 1.9040931204942781


In [6]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime

# Load the data
train_data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

# Label Encoding
label_encoder = LabelEncoder()

# Apply label encoding to all columns
for column in train_data.columns:
    if train_data[column].dtype == 'object':
        train_data[column] = label_encoder.fit_transform(train_data[column])
        test_data[column] = label_encoder.transform(test_data[column])

# Split features and target
X = train_data.drop(['id', 'Rings'], axis=1)
y = train_data['Rings']

# Feature Engineering
X['Volume'] = X['Length'] * X['Diameter'] * X['Height']
test_data['Volume'] = test_data['Length'] * test_data['Diameter'] * test_data['Height']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_data_scaled = scaler.transform(test_data.drop(['id'], axis=1))

# Splitting data for custom cross-validation manually
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize CatBoost model
catboost_model = cb.CatBoostRegressor(iterations=1500,
                                      learning_rate=0.03,
                                      depth=12,
                                      loss_function='RMSE',
                                      verbose=100)

# Train CatBoost model
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

# Predict with CatBoost model
catboost_preds = catboost_model.predict(test_data_scaled)

# Calculate RMSE for CatBoost
catboost_rmse = np.sqrt(mean_squared_error(y_val, catboost_model.predict(X_val)))
print(f'CatBoost RMSE: {catboost_rmse}')

# Create submission file
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'Rings': catboost_preds
})

# Save submission file
submission_df.to_csv('submission_catboost_cross_val.csv', index=False)

0:	learn: 3.1174516	test: 3.1536600	best: 3.1536600 (0)	total: 129ms	remaining: 3m 13s
100:	learn: 1.8925154	test: 1.9494062	best: 1.9494062 (100)	total: 5.65s	remaining: 1m 18s
200:	learn: 1.8220066	test: 1.9076746	best: 1.9076746 (200)	total: 11.3s	remaining: 1m 13s
300:	learn: 1.7777243	test: 1.8929567	best: 1.8929567 (300)	total: 17.3s	remaining: 1m 8s
400:	learn: 1.7390345	test: 1.8858630	best: 1.8858630 (400)	total: 22.9s	remaining: 1m 2s
500:	learn: 1.7032320	test: 1.8809483	best: 1.8809483 (500)	total: 28.6s	remaining: 57s
600:	learn: 1.6688656	test: 1.8777279	best: 1.8777279 (600)	total: 34.1s	remaining: 51.1s
700:	learn: 1.6380580	test: 1.8753931	best: 1.8753931 (700)	total: 40.1s	remaining: 45.7s
800:	learn: 1.6102367	test: 1.8736635	best: 1.8736635 (800)	total: 46s	remaining: 40.1s
900:	learn: 1.5863519	test: 1.8726506	best: 1.8726142 (897)	total: 51.6s	remaining: 34.3s
1000:	learn: 1.5624815	test: 1.8710664	best: 1.8710664 (1000)	total: 57.2s	remaining: 28.5s
1100:	learn: 

In [7]:
submission_df['Rings'] = submission_df['Rings'].round().astype(int)

# Save submission file
submission_df.to_csv('submission_catboost_cross_val_rounded.csv', index=False)

In [8]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime

# Load the data
train_data = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

# Label Encoding
label_encoder = LabelEncoder()

# Apply label encoding to all columns
for column in train_data.columns:
    if train_data[column].dtype == 'object':
        train_data[column] = label_encoder.fit_transform(train_data[column])
        test_data[column] = label_encoder.transform(test_data[column])

# Split features and target
X = train_data.drop(['id', 'Rings'], axis=1)
y = train_data['Rings']

# Feature Engineering
X['Volume'] = X['Length'] * X['Diameter'] * X['Height']
test_data['Volume'] = test_data['Length'] * test_data['Diameter'] * test_data['Height']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_data_scaled = scaler.transform(test_data.drop(['id'], axis=1))

# Splitting data for custom cross-validation manually
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize CatBoost model with optimized hyperparameters
catboost_model = cb.CatBoostRegressor(iterations=3000,
                                      learning_rate=0.02,
                                      depth=10,
                                      loss_function='RMSE',
                                      l2_leaf_reg=3,
                                      od_type='Iter',
                                      od_wait=200,
                                      verbose=200)

# Train CatBoost model
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100)

# Predict with CatBoost model
catboost_preds = catboost_model.predict(test_data_scaled)

# Calculate RMSE for CatBoost
catboost_rmse = np.sqrt(mean_squared_error(y_val, catboost_model.predict(X_val)))
print(f'CatBoost RMSE: {catboost_rmse}')

# Create submission file
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'Rings': catboost_preds
})

# Save submission file
submission_df.to_csv('submission_catboost_improved.csv', index=False)


0:	learn: 3.1345438	test: 3.1707063	best: 3.1707063 (0)	total: 25.4ms	remaining: 1m 16s
200:	learn: 1.8800356	test: 1.9392677	best: 1.9392677 (200)	total: 4.92s	remaining: 1m 8s
400:	learn: 1.8222286	test: 1.9043863	best: 1.9043863 (400)	total: 9.72s	remaining: 1m 3s
600:	learn: 1.7826420	test: 1.8909686	best: 1.8909686 (600)	total: 14.6s	remaining: 58.1s
800:	learn: 1.7489537	test: 1.8820964	best: 1.8820964 (800)	total: 19.3s	remaining: 53s
1000:	learn: 1.7164899	test: 1.8758082	best: 1.8758082 (1000)	total: 24.4s	remaining: 48.7s
1200:	learn: 1.6899979	test: 1.8712128	best: 1.8712128 (1200)	total: 29.2s	remaining: 43.8s
1400:	learn: 1.6651400	test: 1.8674098	best: 1.8673942 (1398)	total: 34.4s	remaining: 39.2s
1600:	learn: 1.6420368	test: 1.8643037	best: 1.8643037 (1600)	total: 39.2s	remaining: 34.2s
1800:	learn: 1.6208511	test: 1.8626963	best: 1.8626963 (1800)	total: 44.1s	remaining: 29.3s
2000:	learn: 1.6016316	test: 1.8615710	best: 1.8615710 (2000)	total: 48.9s	remaining: 24.4s
22

above gave best score 